## DSPy Demo
This notebook demonstrates the of DSPy framework for automatic prompt optimization.

In [1]:
%cd ../

/Users/jakubzovak/Desktop/dspy-intro


In [2]:
%load_ext autoreload
%autoreload 2

## Setup

In [3]:
import json
import os
from pathlib import Path
from typing import Literal, Any, cast

import dspy
from dotenv import load_dotenv
from dspy.adapters.baml_adapter import BAMLAdapter
from sklearn.model_selection import train_test_split
import mlflow

from notebooks.utils.dtos import Merger, Acquisition, Other
from notebooks.utils.evaluation import validate_answer
from notebooks.utils.helpers import extract_first_n_sentences, read_data, get_gt_pydantic_model


Load environment with MlFlow and OpenAI keys.

In [4]:
load_dotenv()

True

## MlFlow Setup

Run MlFlow server for logging traces and tracking optimization runs.

In [5]:
MLFLOW_PORT = os.getenv("MLFLOW_PORT")

In [ ]:
!docker run -d -p {MLFLOW_PORT}:5000 -v $(pwd)/mlruns:/mlflow/mlruns -e MLFLOW_TRACKING_URI=http://0.0.0.0:5000 \
  --name mlflow-server ghcr.io/mlflow/mlflow:v3.7.0 mlflow server \
    --host 0.0.0.0 \
    --port 5000 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root /mlflow/mlruns

4edf854c99ba1be4d8defc9fcf647edc87756f2bf7088e744a09aa18e6bcbf4d


In [6]:
mlflow.set_tracking_uri(f"http://localhost:{MLFLOW_PORT}")

## DSPy

### Adapters
Adapters are basically middleware for communication with LLM. They support different way of interacting with LLM such as structured output.

In [7]:
predictor_lm = dspy.LM(
    model="openrouter/google/gemini-2.5-flash-lite",
    api_base="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

dspy.configure(lm=predictor_lm, adapter=BAMLAdapter()) # type: ignore

Turn on MlFlow tracking with auto-logging for DSPy.

In [8]:
mlflow.set_experiment("DSPy Demo")
mlflow.dspy.autolog(log_compiles=True, log_evals=True, log_traces_from_compile=True)

2025/12/08 22:54:08 WARNING mlflow.utils.autologging_utils: MLflow dspy autologging is known to be compatible with 2.5.41 <= dspy, but the installed version is 3.0.4b1. If you encounter errors during autologging, try upgrading / downgrading dspy to a compatible version, or try upgrading MLflow.


### Signatures
Signatures are declarative specification of the desired behavior that we expect from the LLM.

In [9]:
class ClassifyArticle(dspy.Signature):
    """
    Analyze the news article and classify it as a merger or acquisition deal.
    - A merger is when two companies combine to form a new entity.
    - An acquisition is when one company takes over another.
    - If it mentions a potential, rumored or failed deal, classify it as "Other".
    """

    text: str = dspy.InputField()
    article_type: Literal["merger", "acquisition", "other"] = dspy.OutputField()


class ExtractMergerInfo(dspy.Signature):
    """
    Extract information about companies involved in a merger deal.
    - If the currency symbol is just "$", assume USD.
    """

    text: str = dspy.InputField()
    merger_info: Merger = dspy.OutputField()


class ExtractAcquisitionInfo(dspy.Signature):
    """
    Extract information about companies involved in an acquisition deal.
    - If the currency symbol is just "$", assume USD.
    """

    text: str = dspy.InputField()
    acquisition_info: Acquisition = dspy.OutputField()

### Modules
Modules are core building block in DSPy that combine signatures and logic. Modules are composable and therefore can be combined to form more complex systems.

In [10]:
class Extract(dspy.Module):
    num_sentences: int = 5

    def __init__(self):
        self.classifier = dspy.Predict(ClassifyArticle)
        self.merger_extractor = dspy.Predict(ExtractMergerInfo)
        self.acquisition_extractor = dspy.Predict(ExtractAcquisitionInfo)

    def classify(self, text: str) -> str:
        text = extract_first_n_sentences(text, num_sentences=self.num_sentences)

        result = self.classifier(text=text)
        article_type : str = result.article_type # type: ignore

        return article_type


    def forward(self, text: str, article_id: int) -> Merger | Acquisition | Other:
        article_type = self.classify(text)

        if article_type == "merger":
            extracted_result = self.merger_extractor(text=text)
            merger_info : Merger = extracted_result.merger_info
            merger_info.article_id = article_id
            return merger_info
        
        elif article_type == "acquisition":
            extracted_result = self.acquisition_extractor(text=text)
            acquisition_info : Acquisition = extracted_result.acquisition_info
            acquisition_info.article_id = article_id
            return acquisition_info
        else:
            return Other(article_id=article_id, article_type="other")

## Data Preparation
Load articles about gold mining mergers and acquisitions.

In [11]:
articles = read_data(Path("data/articles.json"))
print(f"Example of loaded article with id {articles[0]['id']}: \n{json.dumps(articles[0], indent=2)}")

Example of loaded article with id 1: 
{
  "id": 1,
  "url": "https://www.australianmining.com.au/elevra-lithium-a-defining-moment-in-north-american-lithium/",
  "title": "Elevra: \u2018A defining moment in North American lithium\u2019",
  "date": "2025-04-09",
  "content": "The company to be created through the merger of ASX-listed Sayona Mining and Piedmont Lithium will be named Elevra Lithium, subject to Sayona shareholder approval. Originally announced in November 2024, the merger is set to create the largest hard-rock lithium producer in the US. The transaction will result in an approximate 50:50 equity holding for Sayona and Piedmont shareholders on a fully diluted basis following the deal\u2019s closing, which is expected in the first half of 2025. Now the merged entity has been named, nominees for the board have been announced. The Elevra Lithium board will initially comprise eight members, including four directors to be appointed by Sayona and four directors to be appointed by 

Load ground truth about the articles which contains their classification and information extraction.

In [12]:
ground_truths: list[dict[str, Any]] = read_data(Path("data/ground_truth.json"))
print(f"Example of loaded ground truth with id {ground_truths[0]['article_id']}: \n{json.dumps(ground_truths[0], indent=2)}")


Example of loaded ground truth with id 1: 
{
  "article_id": 1,
  "company_1": "Sayona Mining",
  "company_1_ticker": null,
  "company_2": "Piedmont Lithium",
  "company_2_ticker": null,
  "merged_entity": "Elevra Lithium",
  "deal_amount": null,
  "deal_currency": "Unknown",
  "article_type": "merger"
}


Convert data to DSPy examples.

In [13]:
NUM_SENTENCES = 5
dspy_examples: list[dspy.Example] = []
for article  in articles:
    ground_truth: dict[str, Any] = ground_truths[article["id"] - 1] # type: ignore
    gt_pydantic_model = get_gt_pydantic_model(ground_truth)
    example = dspy.Example(
        text=(
            article["title"] + "\n" + extract_first_n_sentences(article["content"], NUM_SENTENCES)
        ), 
        article_id=article["id"], 
        expected_output=gt_pydantic_model
    ).with_inputs("text", "article_id") # type: ignore
    dspy_examples.append(example)

Perform train-test split. For DSPy we actually do not need lot of training data.  <br>
In fact, using something **around 20-80 train-test split is quite standard**.

In [14]:
train_set, test_set = cast(
    tuple[list[dspy.Example], list[dspy.Example]],
    train_test_split(dspy_examples, test_size=0.50, random_state=39)
)

print(f"Train set size: {len(train_set)} \nTest set size:  {len(test_set)}")

Train set size: 20 
Test set size:  20


## Run Baseline DSPy Program

Instantiate and test out baseline DSPy program without any optimization.

In [15]:
baseline_extract = Extract()

Execute the baseline module.

In [16]:
extracted_article_1 = baseline_extract(text=train_set[0].text, article_id=train_set[0].article_id)

Trace(trace_id=tr-d37bcbaed93ba7286578db58164a956b)

In [17]:
print(f"Extracted article:\n{extracted_article_1.model_dump_json(indent=2)}")

Extracted article:
{
  "article_id": 27,
  "parent_company": "Blue Canyon Coal",
  "parent_company_ticker": null,
  "child_company": "Highland Basin Resources",
  "child_company_ticker": null,
  "deal_amount": "2.2 bn",
  "deal_currency": "USD",
  "article_type": "acquisition"
}


Explore the underlying prompt. With `n=1` you will see extraction prompt. While with `n=2` you will see classification prompt.

In [ ]:
# dspy.inspect_history(n=1)

Store the baseline extractor as DSPy program in JSON format.

In [18]:
BASELINE_PROGRAM_PATH = "programs/baseline_extractor.json"
baseline_extract.save(BASELINE_PROGRAM_PATH)

## Optimizations

### LabelFewShot Optimizer

In [19]:
label_fewshot_optimizer = dspy.BootstrapFewShotWithRandomSearch(
    metric=validate_answer,
    max_bootstrapped_demos=5,
    max_labeled_demos=10,
    max_rounds=1,
    max_errors=1,
    num_candidate_programs=10,
)

Going to sample between 1 and 5 traces per predictor.
Will attempt to bootstrap 10 candidate sets.


In [20]:
fewshot_extract = label_fewshot_optimizer.compile(baseline_extract, trainset=train_set)

2025/12/08 22:54:44 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '99217d3e58c64222b3fbb7450148c323', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/12/08 22:54:44 WARNING mlflow.dspy.util: Failed to save dspy module state: [Errno 30] Read-only file system: '/mlflow'


Average Metric: 2.88 / 3 (95.8%):  10%|█         | 2/20 [00:01<00:12,  1.44it/s] 

/Users/jakubzovak/Desktop/dspy-intro/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='{\n  "ac...one, 'reasoning': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...finish_reason': 'STOP'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


Average Metric: 17.11 / 20 (85.6%): 100%|██████████| 20/20 [00:03<00:00,  5.44it/s]

2025/12/08 22:54:48 INFO dspy.evaluate.evaluate: Average Metric: 17.11111111111111 / 20 (85.6%)
2025/12/08 22:54:48 WARNING mlflow.dspy.util: Failed to save dspy module state: [Errno 30] Read-only file system: '/mlflow'
2025/12/08 22:54:48 WARNING mlflow.dspy.util: Failed to log dataset: [Errno 30] Read-only file system: '/mlflow'
2025/12/08 22:54:48 WARNING mlflow.dspy.util: Failed to save dspy module state: [Errno 30] Read-only file system: '/mlflow'


...

2025/12/08 22:56:09 INFO dspy.evaluate.evaluate: Average Metric: 18.569444444444443 / 20 (92.8%)



🏃 View run eval_12 at: http://localhost:5530/#/experiments/2/runs/38693ec92e324fbb9675754a2bbdc633
🧪 View experiment at: http://localhost:5530/#/experiments/2


2025/12/08 22:56:09 WARNING mlflow.dspy.util: Failed to save dspy module state: [Errno 30] Read-only file system: '/mlflow'
2025/12/08 22:56:09 WARNING mlflow.dspy.util: Failed to log dataset: [Errno 30] Read-only file system: '/mlflow'


Scores so far: [85.56, 86.25, 89.86, 92.29, 88.19, 83.89, 88.68, 86.25, 91.94, 93.47, 85.69, 90.62, 92.85]
Best score so far: 93.47
13 candidate programs found.
🏃 View run mysterious-eel-200 at: http://localhost:5530/#/experiments/2/runs/99217d3e58c64222b3fbb7450148c323
🧪 View experiment at: http://localhost:5530/#/experiments/2


[Trace(trace_id=tr-fdd134da9dd88ea3deb26e11cfc4e8d4), Trace(trace_id=tr-8895fa7c7c27eba0675b0e6a07fbe7db), Trace(trace_id=tr-28f8cf0e72d9088d6be6900d953c47f8), Trace(trace_id=tr-12243bdae2c3e8ceaf6f19b050190711), Trace(trace_id=tr-923421eafd6cc3f12db8db925dc04adc), Trace(trace_id=tr-064d668f91124ee1c1d7cf4625c41d3b), Trace(trace_id=tr-6ba12774efe2be51eb1e8a3a7bd0a308), Trace(trace_id=tr-8e1dff487f7d324d759c79fe268952d7), Trace(trace_id=tr-9a942f6e8d4d677e64afc28b5e5d8de5), Trace(trace_id=tr-00a6f5300560290a8147741ca0aea67d)]

In [21]:
FEWSHOT_PROGRAM_PATH = "programs/fewshot_extractor.json"
fewshot_extract.save(FEWSHOT_PROGRAM_PATH)

## Evaluation

In [ ]:
models_configs = {
    "baseline": {
        "program_path": BASELINE_PROGRAM_PATH,
    },
    "fewshot": {
        "program_path": FEWSHOT_PROGRAM_PATH,
    }
}

In [ ]:
for model_name, model_config in models_configs.items():
    extract_model = Extract()
    extract_model.load(model_config["program_path"])

    train_result = dspy.Evaluate(
        devset=train_set,
        metric=validate_answer,
        num_threads=1,
        display_progress=False,
        display_table=False,
    )(extract_model)

    test_result = dspy.Evaluate(
        devset=test_set,
        metric=validate_answer,
        num_threads=1,
        display_progress=False,
        display_table=False,
    )(extract_model)

    models_configs[model_name]["train_score"] = train_result["score"]
    models_configs[model_name]["test_score"] = test_result["score"]

2025/12/07 18:33:37 INFO dspy.evaluate.evaluate: Average Metric: 16.86111111111111 / 20 (84.3%)
2025/12/07 18:33:50 INFO dspy.evaluate.evaluate: Average Metric: 15.88888888888889 / 20 (79.4%)
2025/12/07 18:34:03 INFO dspy.evaluate.evaluate: Average Metric: 18.583333333333332 / 20 (92.9%)
2025/12/07 18:34:25 INFO dspy.evaluate.evaluate: Average Metric: 17.59722222222222 / 20 (88.0%)


In [23]:
models_configs

{'baseline': {'program_path': 'programs/baseline_extractor.json',
  'train_score': 92.92,
  'test_score': 79.49},
 'fewshot': {'program_path': 'programs/fewshot_extractor.json',
  'train_score': 94.03,
  'test_score': 82.08}}